In [3]:
import src.get_data as get_data
import src.load_data as load
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import datetime 

from keras.callbacks import History 
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, save_model
from keras.layers import  Flatten, Convolution1D, MaxPooling1D
import time
from sklearn import metrics

WINDOW = 22

# LITERATURE:
# [1] http://cs231n.stanford.edu/reports/2015/pdfs/ashwin_final_paper.pdf

In [4]:
import time
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8


import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [6]:

def build_model(input_shape):
    d = 0.2

    model = Sequential()
    model.add(Convolution1D(input_shape = input_shape, 
                            nb_filter=128,
                            filter_length=2,
                            border_mode='valid',
                            activation='relu',
                            subsample_length=1))    
    model.add(MaxPooling1D(pool_length=2))

    model.add(Convolution1D(input_shape = input_shape, 
                            nb_filter=128,
                            filter_length=2,
                            border_mode='valid',
                            activation='relu',
                            subsample_length=1))
    model.add(MaxPooling1D(pool_length=2))

    model.add(Dropout(d))
    model.add(Flatten())
    model.add(Dense(250))
    model.add(Dropout(d))
    model.add(Activation('relu'))

    model.add(Dense(1))
    model.add(Activation('linear'))
    
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    return model

In [11]:
df = get_data.get_data_frame()

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

all_df = df.copy()

x = all_df[['open', 'low', 'high', 'volume']].copy()

y = all_df['close'].copy()
n_features = x.shape[1]

#x = pd.ewma(x,2)
#y = pd.ewma(y,2)
x[['open', 'low', 'high', 'volume']] = x_scaler.fit_transform(x)

y = y_scaler.fit_transform(y.values.reshape(-1, 1))
x['close'] = y

#X_train, y_train = load.load_data(x, WINDOW, TrainTest = False)
X_train, y_train, X_test, y_test = load.load_data(x, WINDOW, train_size= 0.9, TrainTest = True)

In [13]:
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

model = build_model(input_shape=(WINDOW, n_features))

print('START FIT MODEL...')

start = time.time()

history = History()
history= model.fit(X_train, y_train, validation_data=(X_test, y_test),  batch_size=32, epochs=10,verbose=1,
          callbacks=[history])

end = time.time()



/root/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning:

Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(22, 4), activation="relu", filters=128, kernel_size=2, strides=1, padding="valid")`

/root/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning:

Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`

/root/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning:

Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(22, 4), activation="relu", filters=128, kernel_size=2, strides=1, padding="valid")`

/root/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:

Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`



START FIT MODEL...
Train on 637 samples, validate on 71 samples
Epoch 1/10
637/637 [==============================] - 0s - loss: 0.0086 - acc: 0.0000e+00 - val_loss: 0.0125 - val_acc: 0.0141
Epoch 2/10
637/637 [==============================] - 0s - loss: 9.5275e-04 - acc: 0.0000e+00 - val_loss: 0.0100 - val_acc: 0.0141
Epoch 3/10
637/637 [==============================] - 0s - loss: 6.9410e-04 - acc: 0.0000e+00 - val_loss: 0.0105 - val_acc: 0.0141
Epoch 4/10
637/637 [==============================] - 0s - loss: 7.4330e-04 - acc: 0.0000e+00 - val_loss: 0.0166 - val_acc: 0.0141
Epoch 5/10
637/637 [==============================] - 0s - loss: 6.7232e-04 - acc: 0.0000e+00 - val_loss: 0.0094 - val_acc: 0.0141
Epoch 6/10
637/637 [==============================] - 0s - loss: 6.5805e-04 - acc: 0.0000e+00 - val_loss: 0.0138 - val_acc: 0.0141
Epoch 7/10
637/637 [==============================] - 0s - loss: 7.3573e-04 - acc: 0.0000e+00 - val_loss: 0.0130 - val_acc: 0.0141
Epoch 8/10
637/637 [===

In [14]:
print ('Learning time: ', end-start)

today = time.strftime("_%d_%m_%Y")

#pathModel = "../../models/model_5f_" + typeBlockchain + today +".h5"
#save_model(model, pathModel)
#model = load_model(pathModel)
# one day prediction. get last batch known data (now we didnt need in y value and can predict it)    
lastbatch = np.array(x[-WINDOW:])
pred = model.predict([lastbatch.reshape(1,22, 5)])
pred =  np.array(y_scaler.inverse_transform(pred)) # predicted value

# now we make dataframe and create row names in date

lastDate =str(df.date[df.last_valid_index()]).split('-')
currentData = datetime.date(int(lastDate[0]),int(lastDate[1]),int(lastDate[2])) + datetime.timedelta(1)
predictionDate = pd.date_range(currentData, periods=1)
prediction = pd.DataFrame(pred, columns=["predictionPrice"], index = predictionDate.values)

print (prediction)


plot = 1
plotHictory = 1
if plot:
    trainPredict = model.predict(X_train)
    testPredict = model.predict(X_test)

    trainPredict = y_scaler.inverse_transform(trainPredict)
    trainY = y_scaler.inverse_transform([y_train])

    testPredict = y_scaler.inverse_transform(testPredict)
    testY = y_scaler.inverse_transform([y_test])

    trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
    print('Train Score: %.2f RMSE' % (trainScore))

    testScore = metrics.mean_squared_error(testY[0], testPredict[:,0]) ** .5
    print('Test Score: %.2f RMSE' % (testScore))
    prices = df.close.values.astype('float32')
    prices = prices.reshape(len(prices), 1)
    trainPredictPlot = np.empty_like(prices)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[WINDOW:len(trainPredict)+WINDOW, :] = trainPredict

    testPredictPlot = np.empty_like(prices)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[(len(prices) - testPredict.shape[0]):len(prices), :] = testPredict

    plt.plot(pd.DataFrame(prices, columns=["close"], index=df.index).close, label='Actual')
    plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close, label='Training')
    plt.plot(pd.DataFrame(testPredictPlot, columns=["close"], index=df.index).close, label='Testing')
    plt.legend(loc='best')
    plt.show()

    interactiveGrapth = 1
    if interactiveGrapth:

        Actual = pd.DataFrame(prices, columns=["close"], index=df.index).close
        Training = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close
        Testing = pd.DataFrame(testPredictPlot, columns=["close"], index=df.date).close

        ActualValues = go.Scatter( x = df.date, y = Actual, name = 'ActualValues')
        TrainingValues = go.Scatter( x = df.date, y = Training, name = 'TrainingValues')
        TestingValues = go.Scatter( x = df.date, y = Testing, name = 'PredictedValues')

        iplot([ActualValues,TrainingValues, TestingValues])
        plt.show()

    if plotHictory:

        plt.plot(history.history['loss'], label = 'TrainLoss')
        plt.plot(history.history['val_loss'], label = 'TestLoss')
        plt.legend()
        plt.show()

Learning time:  0.9288733005523682


ValueError: Error when checking : expected conv1d_5_input to have shape (None, 22, 4) but got array with shape (1, 22, 5)